## Reading Data

In [1]:
import os
import cv2

DATADIR = "Dataset"
CATEGORIES = ["flooded", "non-flooded"]
data = []
labels = []
for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
        data.append(img_array)
        labels.append(CATEGORIES.index(category))

## Splitting Data

In [2]:
from sklearn.model_selection import train_test_split
import numpy as np
X_train, X_val , y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)
X_train = np.array(X_train)
X_val = np.array(X_val)
y_train = np.array(y_train)
y_val = np.array(y_val)


/tmp/ipykernel_109577/2828977190.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = np.array(X_train)
/tmp/ipykernel_109577/2828977190.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_val = np.array(X_val)


## Preprocessing

In [3]:
import imgaug.augmenters as iaa
import cv2
import os
import numpy as np
import random

def data_augmentation(data, labels):
    augmented_images = []
    augmented_labels = []
    seqs = []
    # Define the augmentation sequences
    for i in range(2):
        seq = iaa.Sequential([
            iaa.Fliplr(p=random.uniform(0, 1)),
            iaa.Crop(percent=(0, random.uniform(0, 0.1))),
            iaa.GaussianBlur(sigma=random.uniform(0, 3.0)),
            iaa.AdditiveGaussianNoise(scale=(0, random.uniform(0, 0.1*255))),
            iaa.Multiply((random.uniform(0.5, 1.5), random.uniform(0.5, 1.5))),
            iaa.Affine(
                scale={"x": (random.uniform(0.8, 1.2), random.uniform(0.8, 1.2)), "y": (random.uniform(0.8, 1.2), random.uniform(0.8, 1.2))},
                translate_percent={"x": (random.uniform(-0.2, 0.2), random.uniform(-0.2, 0.2)), "y": (random.uniform(-0.2, 0.2), random.uniform(-0.2, 0.2))},
                rotate=(random.uniform(-45, 45), random.uniform(-45, 45)),
                shear=(random.uniform(-16, 16), random.uniform(-16, 16))
            )
        ], random_order=True)
        seqs.append(seq)

    for i, image in enumerate(data):
        if i % 100 == 0:
            print(i)
        augmented_images.append(image)
        augmented_labels.append(labels[i])
        for seq in seqs:
            augmented_image = seq(image=image)
            augmented_images.append(augmented_image)
            augmented_labels.append(labels[i])
    
    return np.array(augmented_images), np.array(augmented_labels)


In [4]:
# apply augmentation on training data
X_train_aug, y_train_aug = data_augmentation(X_train, y_train)

# apply augmentation on validation data
X_val_aug, y_val_aug = data_augmentation(X_val, y_val)

0


: 

: 

## Features Extraction

In [ ]:
def extract_image_features(image):
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (227 , 227))
    return image

X_train_aug = np.array([extract_image_features(image) for image in X_train_aug])
X_val_aug = np.array([extract_image_features(image) for image in X_val_aug])

from keras.utils import to_categorical

y_train_aug = to_categorical(y_train_aug)
y_val_aug = to_categorical(y_val_aug)

## Model

### Model Building

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import (
    Conv2D,
    AveragePooling2D,
    Flatten,
    Dense,
    Activation,
    MaxPool2D,
    BatchNormalization,
    Dropout,
)

from tensorflow.keras.utils import to_categorical

In [ ]:
# model = Sequential()
# # 1st layer (CONV + pool + batchnorm)
# model.add(
#     Conv2D(
#         filters=96,
#         kernel_size=(11, 11),
#         strides=(4, 4),
#         padding="valid",
#         input_shape=(227, 227, 3),
#     )
# )
# model.add(Activation("relu"))
# model.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
# model.add(BatchNormalization())
# # 2nd layer (CONV + pool + batchnorm)
# model.add(
#     Conv2D(
#         filters=256,
#         kernel_size=(5, 5),
#         strides=(1, 1),
#         padding="same",
#         kernel_regularizer=l2(0.0005),
#     )
# )
# model.add(Activation("relu"))
# model.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding="valid"))
# model.add(BatchNormalization())
# # layer 3 (CONV + batchnorm)
# model.add(
#     Conv2D(
#         filters=384,
#         kernel_size=(3, 3),
#         strides=(1, 1),
#         padding="same",
#         kernel_regularizer=l2(0.0005),
#     )
# )
# model.add(Activation("relu"))
# model.add(BatchNormalization())
# # layer 4 (CONV + batchnorm)
# model.add(
#     Conv2D(
#         filters=384,
#         kernel_size=(3, 3),
#         strides=(1, 1),
#         padding="same",
#         kernel_regularizer=l2(0.0005),
#     )
# )
# model.add(Activation("relu"))
# model.add(BatchNormalization())
# # layer 5 (CONV + batchnorm)
# model.add(
#     Conv2D(
#         filters=256,
#         kernel_size=(3, 3),
#         strides=(1, 1),
#         padding="same",
#         kernel_regularizer=l2(0.0005),
#     )
# )
# model.add(Activation("relu"))
# model.add(BatchNormalization())
# model.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding="valid"))
# model.add(Flatten())
# # layer 6 (Dense layer + dropout)
# model.add(Dense(units=4096, activation="relu"))
# model.add(Dropout(0.5))
# # layer 7 (Dense layers)
# model.add(Dense(units=4096, activation="relu"))
# model.add(Dropout(0.5))
# # layer 8 (softmax output layer)
# model.add(Dense(units=2, activation="softmax"))


In [ ]:
model = Sequential()
# 1st layer (CONV + pool + batchnorm)
model.add(
    Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides=(2, 2),
        padding="same",
        input_shape=(227, 227, 3),
    )
)
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(BatchNormalization())
# 2nd layer (CONV + pool + batchnorm)
model.add(
    Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding="same",
        kernel_regularizer=l2(0.0005),
    )
)
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2, 2), padding="valid"))
model.add(BatchNormalization())
# layer 3 (CONV + batchnorm)
model.add(
    Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding="same",
        kernel_regularizer=l2(0.0005),
    )
)
model.add(Activation("relu"))
model.add(BatchNormalization())
# layer 4 (CONV + batchnorm)
model.add(
    Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding="same",
        kernel_regularizer=l2(0.0005),
    )
)
model.add(Activation("relu"))
model.add(BatchNormalization())
# layer 5 (CONV + batchnorm)
model.add(
    Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides=(1, 1),
        padding="same",
        kernel_regularizer=l2(0.0005),
    )
)
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2), padding="valid"))
model.add(Flatten())
# layer 6 (Dense layer + dropout)
model.add(Dense(units=1024, activation="relu"))
model.add(Dropout(0.5))
# layer 7 (Dense layers)
model.add(Dense(units=1024, activation="relu"))
model.add(Dropout(0.5))
# layer 8 (softmax output layer)
model.add(Dense(units=2, activation="softmax"))

### Model Training

In [ ]:
model.summary()
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=7, validation_data=(X_val, y_val))

## Evaluation

In [ ]:
# Get confusion matrix , f1 score
# Plot confusion matrix
from sklearn.metrics import confusion_matrix, f1_score
import seaborn as sns
import pandas as pd

y_pred = model.predict(X_val)
y_pred = np.argmax(y_pred, axis=1)
cm = confusion_matrix(y_val, y_pred)
cm_df = pd.DataFrame(cm, index=CATEGORIES, columns=CATEGORIES)
sns.heatmap(cm_df, annot=True, cmap="Blues", fmt="d")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# Plot f1 score
f1 = f1_score(y_val, y_pred, average=None)
f1_df = pd.DataFrame(f1, index=CATEGORIES, columns=["F1 Score"])
f1_df.plot(kind="bar", ylim=(0, 1))
plt.title("F1 Score")
plt.show()


